In [1]:
#Load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
# NBA API
#from nba_api.stats.static import players
#from nba_api.stats.endpoints import commonplayerinfo
#from nba_api.stats.static import teams

# Make player Metrics

## Metric 1: Overall player metric 

We have 
- True shooting decile, 
- Points per minute decile, 
- rebounding decile, 
- 3pt decile, 
- ast decile
- 3pt D decile
- inside 10ft decile

How to weigh them?


In [2]:
# Data
df_21 = pd.read_excel('final_player_data\\deciles_alldata20.xlsx')
df_20 = pd.read_excel('final_player_data\\deciles_alldata19.xlsx')
df_19 = pd.read_excel('final_player_data\\deciles_alldata18.xlsx')
df_18 = pd.read_excel('final_player_data\\deciles_alldata17.xlsx')
df_17 = pd.read_excel('final_player_data\\deciles_alldata16.xlsx')


In [3]:
df_20.describe().round(1)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x,Unnamed: 1_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,...,ins_10_D_FGA,ins_10_D_FG%,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TS_decile
count,366.0,366.0,366.0,366.0,366.0,366.0,366.0,366.0,366.0,366.0,...,366.0,366.0,366.0,366.0,366.0,366.0,366.0,366.0,366.0,366.0
mean,182.5,187.8,187.8,183.1,26.6,54.2,27.4,26.8,24.4,110.1,...,4.7,56.8,4.5,4.5,4.5,4.5,4.4,4.5,4.4,4.5
std,105.8,112.8,112.8,113.6,4.2,12.3,10.8,8.5,6.7,4.8,...,1.7,4.0,2.9,2.9,2.9,2.9,2.9,2.9,2.9,2.9
min,0.0,1.0,1.0,1.0,19.0,20.0,3.0,9.0,9.6,97.0,...,1.5,45.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,91.2,92.2,92.2,89.0,23.0,46.0,19.0,20.0,18.9,106.6,...,3.5,54.1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
50%,182.5,183.5,183.5,180.0,26.0,56.0,28.0,26.0,24.2,110.1,...,4.6,56.9,4.0,4.0,4.0,4.0,4.0,4.5,4.5,4.0
75%,273.8,274.8,274.8,268.0,29.0,64.0,36.0,32.0,30.3,113.7,...,5.6,59.4,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
max,365.0,435.0,435.0,435.0,37.0,72.0,52.0,53.0,37.6,121.7,...,10.9,70.2,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0


In [4]:
df_21.shape

(377, 75)

In [5]:
d21 = df_21.T.drop_duplicates().T
d20 = df_20.T.drop_duplicates().T
d19 = df_19.T.drop_duplicates().T
d18 = df_18.T.drop_duplicates().T
d17 = df_17.T.drop_duplicates().T
d21.shape

(377, 67)

In [6]:
da_ds = [d21, d20, d19, d18, d17]

In [7]:
#Check our decile options
pd20 = df_20[d20.columns[d20.columns.str.contains("decile")]]
pd20

,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TS_decile
0,8,9,4,3,3,6,4,1
1,7,3,2,7,2,3,5,6
2,3,4,6,8,2,2,3,8
3,5,2,8,1,1,2,0,0
4,4,6,6,1,9,1,5,8
...,...,...,...,...,...,...,...,...
361,7,1,3,7,5,4,1,4
362,6,9,9,5,8,6,6,4
363,6,0,0,3,4,3,2,7
364,2,0,6,9,6,0,0,9


In [9]:
decile_cols = pd20.columns
decile_cols = list(decile_cols)
decile_cols

['usg_decile',
 '3pt_Def_decile',
 'Def_within10_decile',
 '3pt_decile',
 'Reb_%_decile',
 'AST_%_decile',
 'TOV_%_decile',
 'TS_decile']

In [10]:
colz = ['PLAYER',
 'TEAM_x',
 'MIN_x',
 'TS_decile',
 'usg_decile',
 '3pt_Def_decile',
 'Def_within10_decile',
 '3pt_decile',
 'Reb_%_decile',
 'AST_%_decile',
 'TOV_%_decile'] 

p21= d21[colz]

p21

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile
0,Deni Avdija,WAS,24.2,2,3,1,1,3,6,4,5
1,Dwight Powell,DAL,21.9,9,1,3,7,5,7,2,5
2,Kevon Looney,GSW,21.1,6,0,3,3,0,9,5,6
3,Mikal Bridges,PHX,34.8,9,2,3,5,7,2,2,0
4,Saddiq Bey,DET,33,2,7,6,4,5,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...
372,T.J. McConnell,IND,24.2,1,3,4,6,2,4,9,2
373,Derrick Rose,NYK,24.5,2,7,0,0,8,1,8,4
374,Nerlens Noel,NYK,22.5,6,0,9,1,0,8,0,9
375,Brandon Williams,POR,26.7,0,8,9,4,1,1,8,8


In [11]:
p21['min_decile'] = pd.qcut(p21['MIN_x'], 10, labels=False)

<ipython-input-11-4f9263b04aa6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p21['min_decile'] = pd.qcut(p21['MIN_x'], 10, labels=False)


In [12]:
p21['trav_score'] = ((p21['TS_decile'] * .14) + 
                        (p21['3pt_decile'] * .14) + 
                        (p21['AST_%_decile'] * .08) -
                        (p21['TOV_%_decile'] * .08) +                   # negative 3 percent
                        # Defensive Metrics 
                        ((9 - p21['3pt_Def_decile']) * .17) +
                        ((9 - p21['Def_within10_decile']) * .17) +
                        ((9 - p21['Reb_%_decile']) * .10) +
                        # Minutes played and usage
                        (p21['usg_decile'] * .04) +                   
                        (p21['MIN_x'] * .08))                           # players who play more minutes play against better players


p21

<ipython-input-12-cdc44ca19baa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p21['trav_score'] = ((p21['TS_decile'] * .14) +


,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,min_decile,trav_score
0,Deni Avdija,WAS,24.2,2,3,1,1,3,6,4,5,5,5.696
1,Dwight Powell,DAL,21.9,9,1,3,7,5,7,2,5,4,5.072
2,Kevon Looney,GSW,21.1,6,0,3,3,0,9,5,6,3,4.488
3,Mikal Bridges,PHX,34.8,9,2,3,5,7,2,2,0,9,7.664
4,Saddiq Bey,DET,33,2,7,6,4,5,5,5,0,8,6.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,T.J. McConnell,IND,24.2,1,3,4,6,2,4,9,2,5,4.896
373,Derrick Rose,NYK,24.5,2,7,0,0,8,1,8,4,5,7.82
374,Nerlens Noel,NYK,22.5,6,0,9,1,0,8,0,9,4,3.38
375,Brandon Williams,POR,26.7,0,8,9,4,1,1,8,8,6,4.246


In [13]:
for d in da_ds: 
    d['trav_score'] = ((d['TS_decile'] * .14) + 
                        (d['3pt_decile'] * .14) + 
                        (d['AST_%_decile'] * .08) -
                        (d['TOV_%_decile'] * .08) +                   # negative 3 percent
                        # Defensive Metrics 
                        ((9 - d['3pt_Def_decile']) * .17) +
                        ((9 - d['Def_within10_decile']) * .17) +
                        ((9 - d['Reb_%_decile']) * .10) +
                        # Minutes played and usage
                        (d['usg_decile'] * .04) +                   
                        (d['MIN_x'] * .08))                           # players who play more minutes play against better players



In [14]:
p21.sort_values('trav_score', ascending=False, inplace= True)
p21.head(25)

<ipython-input-14-6ac605689c73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p21.sort_values('trav_score', ascending=False, inplace= True)


,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,min_decile,trav_score
370,Kyrie Irving,BKN,37.6,7,9,5,2,9,1,8,2,9,8.758
55,Tyrese Maxey,PHI,35.3,7,6,4,4,9,0,7,0,9,8.464
241,LeBron James,LAL,37.2,8,9,3,0,6,7,9,6,9,8.286
35,Desmond Bane,MEM,29.8,7,7,3,1,9,3,5,1,7,8.204
182,Stephen Curry,GSW,34.5,7,9,1,3,7,4,9,6,9,8.2
364,Klay Thompson,GSW,29.4,3,9,1,1,8,2,6,0,7,8.152
251,Kevin Durant,BKN,37.2,9,9,5,1,7,6,8,6,9,8.076
164,Fred VanVleet,TOR,37.9,4,8,4,2,7,1,8,4,9,8.052
229,Eric Gordon,HOU,29.3,8,5,2,2,9,0,5,8,7,7.964
123,Devin Booker,PHX,34.5,5,9,2,3,7,4,7,2,9,7.91


In [15]:
p21.trav_score = p21.trav_score.astype(str).astype(np.float64)

c:\Users\tmcro\anaconda3\envs\learn-env\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
group = p21.groupby('TEAM_x')['trav_score'].agg(['sum', 'mean', 'count'])

In [17]:
group

,sum,mean,count
TEAM_x,,,
ATL,61.612,5.601091,11
BKN,71.188,5.476000,13
BOS,59.068,5.906800,10
CHA,58.376,5.837600,10
CHI,64.108,5.342333,12
CLE,67.378,5.614833,12
DAL,64.110,5.342500,12
DEN,68.094,5.238000,13
DET,60.296,5.024667,12


In [18]:
por = p21.loc[p21['TEAM_x']== "POR"]
por

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,min_decile,trav_score
369,Damian Lillard,POR,36.4,3,9,5,6,3,1,9,4,9,6.502
337,Nassir Little,POR,25.9,5,2,0,1,4,6,1,3,5,6.442
228,Anfernee Simons,POR,29.5,6,8,8,9,9,0,7,5,7,6.010
327,Joe Ingles,POR,24.9,3,1,0,9,5,1,7,4,5,5.722
265,Josh Hart,POR,33.2,8,5,7,7,5,7,7,7,8,5.556
259,Eric Bledsoe,POR,25.2,1,6,5,2,2,3,8,9,5,5.066
352,Keon Johnson,POR,18.8,0,6,0,9,4,1,6,7,2,4.554
175,Ben McLemore,POR,20.1,5,6,8,9,6,0,2,2,3,4.458
375,Brandon Williams,POR,26.7,0,8,9,4,1,1,8,8,6,4.246
240,Jusuf Nurkic,POR,28.2,6,8,9,4,1,9,6,9,6,4.166


In [19]:
bos = p21.loc[p21['TEAM_x']== "BOS"]
bos

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,min_decile,trav_score
39,Jayson Tatum,BOS,35.9,6,9,2,1,5,7,7,4,9,7.762
150,Jaylen Brown,BOS,33.6,5,9,4,3,6,5,6,5,8,6.938
97,Payton Pritchard,BOS,14.1,6,5,2,2,9,3,7,1,0,6.888
109,Al Horford,BOS,29.1,5,2,3,0,4,8,6,1,7,6.718
208,Robert Williams III,BOS,29.6,9,0,1,0,0,9,3,6,7,6.278
28,Grant Williams,BOS,24.4,9,0,6,2,9,4,0,6,5,6.192
48,Derrick White,BOS,29.3,3,6,7,1,2,1,8,3,7,6.184
92,Marcus Smart,BOS,32.3,2,4,2,7,4,1,8,7,8,5.994
316,Daniel Theis,BOS,20.8,7,3,3,5,2,7,1,7,3,4.464
272,Aaron Nesmith,BOS,11,1,3,9,6,1,4,0,8,0,1.650


### Sort by TScore, Filter out by Min, Games Played

In [22]:
d21

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x,Unnamed: 1_x,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,...,ins_10_D_FG%,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,trav_score
0,0,0,1,1,Deni Avdija,WAS,21,82,35,47,...,52.2,2,3,1,1,3,6,4,5,5.696
1,1,1,2,1,Dwight Powell,DAL,30,82,52,30,...,59.8,9,1,3,7,5,7,2,5,5.072
2,2,2,3,1,Kevon Looney,GSW,26,82,53,29,...,56.1,6,0,3,3,0,9,5,6,4.488
3,3,3,4,1,Mikal Bridges,PHX,25,82,64,18,...,58.3,9,2,3,5,7,2,2,0,7.664
4,4,4,5,1,Saddiq Bey,DET,23,82,23,59,...,56.7,2,7,6,4,5,5,5,0,6.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,372,414,415,414,T.J. McConnell,IND,30,27,9,18,...,59.2,1,3,4,6,2,4,9,2,4.896
373,373,416,417,417,Derrick Rose,NYK,33,26,11,15,...,45.6,2,7,0,0,8,1,8,4,7.82
374,374,421,422,420,Nerlens Noel,NYK,28,25,10,15,...,52,6,0,9,1,0,8,0,9,3.38
375,375,422,423,423,Brandon Williams,POR,22,24,2,22,...,56.5,0,8,9,4,1,1,8,8,4.246


In [27]:
dfilt21 = d21.dropna()
dfilt21 = dfilt21.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 1_x'])
dfilt21 = dfilt21.loc[dfilt21['MIN_x'] > 15]
dfilt21 = dfilt21.loc[:,~dfilt21.columns.duplicated()]
dfilt21 = dfilt21.loc[dfilt21['GP_x'] > 30]
dfilt21 = dfilt21.sort_values('trav_score', ascending = False)
dfilt21.head()


,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,DEFRTG,NETRTG,...,ins_10_D_FG%,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,trav_score
55,Tyrese Maxey,PHI,21,75,45,30,35.3,113.3,109.7,3.6,...,57.3,7,6,4,4,9,0,7,0,8.464
241,LeBron James,LAL,37,56,25,31,37.2,110.9,113.5,-2.5,...,51.8,8,9,3,0,6,7,9,6,8.286
35,Desmond Bane,MEM,23,76,53,23,29.8,114.7,107.8,6.9,...,52,7,7,3,1,9,3,5,1,8.204
182,Stephen Curry,GSW,34,64,45,19,34.5,114.1,103.4,10.7,...,55.6,7,9,1,3,7,4,9,6,8.2
364,Klay Thompson,GSW,32,32,17,15,29.4,111.4,108,3.3,...,53.4,3,9,1,1,8,2,6,0,8.152


In [29]:
dfilt20 = d20.dropna()
dfilt20 = dfilt20.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1_x'])
dfilt20 = dfilt20.loc[dfilt20['MIN_x'] > 15]
dfilt20 = dfilt20.loc[:,~dfilt20.columns.duplicated()]
dfilt20 = dfilt20.loc[dfilt20['GP_x'] > 30]
dfilt20 = dfilt20.sort_values('trav_score', ascending = False)
dfilt20.head()

,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,DEFRTG,NETRTG,...,ins_10_D_FG%,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TS_decile,trav_score
335,Kevin Durant,BKN,32,35,23,12,33.1,121.7,111.1,10.7,...,50.8,9,1,0,9,7,8,7,9,8.698
283,LeBron James,LAL,36,45,30,15,33.4,112.4,103.2,9.2,...,51,9,0,0,5,7,9,7,7,8.132
288,Bogdan Bogdanovic,ATL,28,44,30,14,29.7,114.9,111,3.9,...,51.7,6,6,1,9,2,7,1,8,8.046
170,Zach LaVine,CHI,26,58,26,32,35.1,112.4,113.1,-0.8,...,55.2,9,4,3,9,4,8,7,8,7.998
338,Shai Gilgeous-Alexander,OKC,22,35,16,19,33.7,107.2,114.9,-7.7,...,57.9,9,2,6,9,3,9,6,8,7.976


In [31]:
dfilt19 = d19.dropna()
dfilt19 = dfilt19.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1_x'])
dfilt19 = dfilt19.loc[dfilt19['MIN_x'] > 15]
dfilt19 = dfilt19.loc[:,~dfilt19.columns.duplicated()]
dfilt19 = dfilt19.loc[dfilt19['GP_x'] > 30]
dfilt19 = dfilt19.sort_values('trav_score', ascending = False)
dfilt19.head()

,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,DEFRTG,NETRTG,...,ins_10_D_FG%,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TS_decile,trav_score
234,Fred VanVleet,TOR,26,54,40,14,35.7,109.1,104.3,4.8,...,54.3,7,3,3,8,1,8,3,4,8.056
132,Khris Middleton,MIL,28,62,46,16,29.9,113.3,100.9,12.4,...,49.8,8,5,0,9,6,8,3,8,8.002
72,Damian Lillard,POR,29,66,33,33,37.5,115.7,114.3,1.4,...,60.6,9,5,8,8,1,9,2,9,7.95
240,LaMarcus Aldridge,SAS,34,53,23,30,33.1,110.2,113.4,-3.2,...,50.6,8,2,1,8,7,5,0,5,7.938
160,Zach LaVine,CHI,25,60,20,40,34.8,106.8,111.2,-4.4,...,52.2,9,3,1,7,3,8,7,5,7.884


In [33]:
dfilt18 = d18.dropna()
dfilt18 = dfilt18.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1_x'])
dfilt18 = dfilt18.loc[dfilt18['MIN_x'] > 15]
dfilt18 = dfilt18.loc[:,~dfilt18.columns.duplicated()]
dfilt18 = dfilt18.loc[dfilt18['GP_x'] > 30]
dfilt18 = dfilt18.sort_values('trav_score', ascending = False)
dfilt18.head()

,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,DEFRTG,NETRTG,...,ins_10_D_FG%,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TS_decile,trav_score
70,Kevin Durant,GSW,30,78,54,24,34.6,118.7,107.3,11.4,...,50.8,9,1,0,5,5,8,5,9,8.618
174,Spencer Dinwiddie,BKN,26,68,36,32,28.1,107.1,109.8,-2.7,...,42.9,8,0,0,4,0,9,6,7,8.308
269,LeBron James,LAL,34,55,28,27,35.2,109.6,107.6,2,...,51,9,0,0,4,7,9,5,7,8.296
284,Derrick Rose,MIN,30,51,24,27,27.3,111.5,110.8,0.7,...,54.3,9,2,2,7,0,8,0,5,8.144
85,Paul George,OKC,29,77,46,31,36.9,112.1,103.9,8.2,...,57.5,9,1,5,8,6,6,3,7,7.992


In [34]:
dfilt17 = d17.dropna()
dfilt17 = dfilt17.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1_x'])
dfilt17 = dfilt17.loc[dfilt17['MIN_x'] > 15]
dfilt17 = dfilt17.loc[:,~dfilt17.columns.duplicated()]
dfilt17 = dfilt17.loc[dfilt17['GP_x'] > 30]
dfilt17 = dfilt17.sort_values('trav_score', ascending = False)
dfilt17.head()

,PLAYER,TEAM_x,AGE_x,GP_x,W_x,L_x,MIN_x,OFFRTG,DEFRTG,NETRTG,...,ins_10_D_FG%,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TS_decile,trav_score
141,Klay Thompson,GSW,28,73,55,18,34.3,112.6,105.6,7.1,...,50.1,8,5,0,9,2,4,1,8,8.594
76,Kyle Lowry,TOR,32,78,57,21,32.2,113.1,105.5,7.6,...,52,6,2,1,8,5,9,4,8,8.406
66,Paul George,OKC,28,79,47,32,36.6,109.7,106,3.7,...,52.9,8,1,2,8,4,6,5,6,8.338
194,Kevin Durant,GSW,29,68,49,19,34.2,115.9,108.9,7,...,50.8,9,6,0,9,6,8,5,9,8.196
156,James Harden,HOU,28,72,59,13,35.4,116.2,106.2,10,...,54.6,9,2,3,6,5,9,7,9,8.062


In [41]:
dfilt21.to_excel('final_player_data\\All Player Data With Deciles - Filtered 2021.xlsx')
dfilt20.to_excel('final_player_data\\All Player Data With Deciles - Filtered 2020.xlsx')
dfilt19.to_excel('final_player_data\\All Player Data With Deciles - Filtered 2019.xlsx')
dfilt18.to_excel('final_player_data\\All Player Data With Deciles - Filtered 2018.xlsx')
dfilt17.to_excel('final_player_data\\All Player Data With Deciles - Filtered 2017.xlsx')

### Top 25 players by Year

Note that this is by relative efficiency in different areas of basketball

In [35]:
colz1 = colz + ['TEAM_x', 'trav_score']

In [36]:
dfilt21[colz1].head(25)

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TEAM_x,trav_score
55,Tyrese Maxey,PHI,35.3,7,6,4,4,9,0,7,0,PHI,8.464
241,LeBron James,LAL,37.2,8,9,3,0,6,7,9,6,LAL,8.286
35,Desmond Bane,MEM,29.8,7,7,3,1,9,3,5,1,MEM,8.204
182,Stephen Curry,GSW,34.5,7,9,1,3,7,4,9,6,GSW,8.2
364,Klay Thompson,GSW,29.4,3,9,1,1,8,2,6,0,GSW,8.152
251,Kevin Durant,BKN,37.2,9,9,5,1,7,6,8,6,BKN,8.076
164,Fred VanVleet,TOR,37.9,4,8,4,2,7,1,8,4,TOR,8.052
229,Eric Gordon,HOU,29.3,8,5,2,2,9,0,5,8,HOU,7.964
123,Devin Booker,PHX,34.5,5,9,2,3,7,4,7,2,PHX,7.91
106,Luke Kennard,LAC,27.4,8,3,3,3,9,1,4,1,LAC,7.772


In [37]:
dfilt20[colz1].head(25)

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TEAM_x,trav_score
335,Kevin Durant,BKN,33.1,9,9,1,0,9,7,8,7,BKN,8.698
283,LeBron James,LAL,33.4,7,9,0,0,5,7,9,7,LAL,8.132
288,Bogdan Bogdanovic,ATL,29.7,8,6,6,1,9,2,7,1,ATL,8.046
170,Zach LaVine,CHI,35.1,8,9,4,3,9,4,8,7,CHI,7.998
338,Shai Gilgeous-Alexander,OKC,33.7,8,9,2,6,9,3,9,6,OKC,7.976
249,Alec Burks,NYK,25.6,4,6,0,0,9,5,5,1,NYK,7.888
289,Gordon Hayward,CHA,34,6,8,4,2,9,5,7,4,CHA,7.82
10,RJ Barrett,NYK,34.9,2,7,1,1,8,4,5,4,NYK,7.772
113,Stephen Curry,GSW,34.2,9,9,5,5,9,4,9,6,GSW,7.716
239,Derrick Rose,NYK,25.6,3,8,1,2,7,1,9,2,NYK,7.678


In [38]:
dfilt19[colz1].head(25)

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TEAM_x,trav_score
234,Fred VanVleet,TOR,35.7,4,7,3,3,8,1,8,3,TOR,8.056
132,Khris Middleton,MIL,29.9,8,8,5,0,9,6,8,3,MIL,8.002
72,Damian Lillard,POR,37.5,9,9,5,8,8,1,9,2,POR,7.95
240,LaMarcus Aldridge,SAS,33.1,5,8,2,1,8,7,5,0,SAS,7.938
160,Zach LaVine,CHI,34.8,5,9,3,1,7,3,8,7,CHI,7.884
123,Anthony Davis,LAL,34.4,8,9,1,0,3,8,6,3,LAL,7.882
29,CJ McCollum,POR,36.5,3,8,5,2,7,1,7,0,POR,7.87
54,James Harden,HOU,36.5,9,9,3,3,5,5,9,7,HOU,7.84
181,Kyle Lowry,TOR,36.2,7,7,2,3,5,3,9,7,TOR,7.826
210,Kemba Walker,BOS,31.1,6,9,5,3,7,2,8,2,BOS,7.548


In [39]:
dfilt18[colz1].head(25)

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TEAM_x,trav_score
70,Kevin Durant,GSW,34.6,9,9,1,0,5,5,8,5,GSW,8.618
174,Spencer Dinwiddie,BKN,28.1,7,8,0,0,4,0,9,6,BKN,8.308
269,LeBron James,LAL,35.2,7,9,0,0,4,7,9,5,LAL,8.296
284,Derrick Rose,MIN,27.3,5,9,2,2,7,0,8,0,MIN,8.144
85,Paul George,OKC,36.9,7,9,1,5,8,6,6,3,OKC,7.992
13,Monte Morris,DEN,24,6,4,3,2,9,0,8,0,DEN,7.93
22,Bojan Bogdanovic,IND,31.8,8,7,4,2,9,3,3,3,IND,7.844
152,Mike Conley,MEM,33.5,6,9,8,1,6,1,9,0,MEM,7.77
160,Stephen Curry,GSW,33.8,9,9,5,5,9,4,8,4,GSW,7.764
156,Darius Miller,NOP,25.5,4,1,0,0,7,0,4,2,NOP,7.74


In [40]:
dfilt17[colz1].head(25)

,PLAYER,TEAM_x,MIN_x,TS_decile,usg_decile,3pt_Def_decile,Def_within10_decile,3pt_decile,Reb_%_decile,AST_%_decile,TOV_%_decile,TEAM_x,trav_score
141,Klay Thompson,GSW,34.3,8,8,5,0,9,2,4,1,GSW,8.594
76,Kyle Lowry,TOR,32.2,8,6,2,1,8,5,9,4,TOR,8.406
66,Paul George,OKC,36.6,6,8,1,2,8,4,6,5,OKC,8.338
194,Kevin Durant,GSW,34.2,9,9,6,0,9,6,8,5,GSW,8.196
156,James Harden,HOU,35.4,9,9,2,3,6,5,9,7,HOU,8.062
303,Tyreke Evans,MEM,30.9,5,9,2,2,8,5,9,3,MEM,7.912
153,Al Horford,BOS,31.6,6,5,1,2,9,7,8,4,BOS,7.898
94,Courtney Lee,NYK,30.4,5,3,2,2,8,1,5,1,NYK,7.872
248,Jimmy Butler,MIN,36.7,7,8,3,4,5,4,7,0,MIN,7.866
107,Giannis Antetokounmpo,MIL,36.7,8,9,2,0,2,8,8,4,MIL,7.836
